In [1]:
# EDA for recipe databases

In [2]:
# --> Dataset: Recipe NLG: https://www.kaggle.com/datasets/paultimothymooney/recipenlg

In [3]:
# EDA Tasks
# 0. Overall understanding of the structure of the dataset
# 1. Count of recipes
# 2. Avg # of ingredients per recipe
# 3. # of unique ingredients
# 4. List of unique ingredients

In [4]:
import csv
import re
from collections import Counter

import pandas as pd

from utils import *

In [5]:
common_food_items = [
    "salt",
    "kosher salt",
    "onion powder",
    "garlic",
    "garlic powder",
    "garlic salt",
    "freshly ground black pepper",
    "fresh ground black pepper",
    "white pepper",
    "freshly ground pepper",
    "ground pepper",
    "fresh ground pepper",
    "ground red pepper",
    "black peppercorns",
    "lemon pepper",
    "butter",
    "salted butter",
    "unsalted butter",
    "oleo",
    "oil",
    "olive oil",
    "vegetable oil",
    "extra-virgin olive oil",
    "extra virgin olive oil",
    "cooking oil",
    "salad oil",
    "cooking spray",
    "flour",
    "whole wheat flour",
    "light brown sugar",
    "dressing",
    "italian dressing",
    "italian salad dressing",
    "+",
    "'s",
    "confectioners sugar",
    "miracle",
    "salad dressing",
    "italian dressing",
    "t",
    "salad",
    "mayo",
    "apple cider vinegar",
    "granulated sugar",
    "poultry seasoning",
    "sugar",
    "brown sugar",
    "white sugar",
    "honey",
    "maple syrup",
    "tarragon",
    "balsamic vinegar",
    "red wine vinegar",
    "white wine vinegar",
    "white vinegar",
    "cider vinegar",
    "soy sauce",
    "worcestershire sauce",
    "dry mustard",
    "rice vinegar",
    "dijon mustard",
    "ketchup",
    "catsup",
    "mayonnaise",
    "hot sauce",
    "mustard",
    "barbecue sauce",
    "pepper",
    "paprika",
    "cumin",
    "cilantro",
    "thyme",
    "cayenne pepper",
    "chili powder",
    "oregano",
    "rosemary",
    "coriander",
    "cinnamon",
    "ground cinnamon",
    "nutmeg",
    "ginger",
    "allspice",
    "tumeric",
    "ground cumin",
    "bay leaf",
    "ground ginger",
    "ground nutmeg",
    "italian seasoning",
    "curry powder",
    "bay leaves",
    "dill",
    "tumeric",
    "beef broth",
    "basil",
    "parsley",
    "marjoram",
    "sage",
    "turmeric",
    "ground coriander",
    "parsley flakes",
    "cayenne",
    "celery salt",
    "curry",
    "ground turmeric",
    "onion salt",
    "cloves",
    "ground cloves",
    "mustard powder",
    "vegetable broth",
    "vanilla extract",
    "vanilla",
    "baking powder",
    "baking soda",
    "black pepper",
    "cornstarch",
    "water",
    "boiling water",
    "cold water",
    "warm water",
    "\\u00bc",
    "all purpose",
    "all-purpose",
    "lemon juice",
    "margarine",
    "powdered sugar",
    "vinegar",
    "shortening",
    "egg yolks",
    "egg whites",
    "ground black pepper",
    "canola oil",
    "fresh cilantro",
    "fresh basil",
    "yeast",
    "marshmallows",
    "whipping cream",
    "clove garlic",
    "chicken broth",
    "chicken stock",
    "vegetable stock",
    "tabasco sauce",
    "red",
    "ground allspice",
    "skinless",
]

In [6]:
pc = ProjectConfig()
original_csv = pc.data_root_dir.joinpath("RecipeDatabase/RecipeNLG_dataset.csv")
if not original_csv.exists():
    print(f"File {original_csv} does not exist. trying local directory")
    original_csv = Path("RecipeNLG_dataset.csv")
    if not original_csv.exists():
        print(f"File {original_csv} does not exist. Exiting")
        raise FileNotFoundError
else:
    df = pd.read_csv(original_csv)

No configuration file found. Using defaults.


In [7]:
df.head()

,Unnamed: 0,title,ingredients,directions,link,source,NER
0,0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu..."
1,1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"[""beef"", ""chicken breasts"", ""cream of mushroom..."
2,2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"[""frozen corn"", ""cream cheese"", ""butter"", ""gar..."
3,3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,Gathered,"[""chicken"", ""chicken gravy"", ""cream of mushroo..."
4,4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...",www.cookbooks.com/Recipe-Details.aspx?id=659239,Gathered,"[""peanut butter"", ""graham cracker crumbs"", ""bu..."


In [8]:
len(df)

2231142

In [9]:
keywords = [
    "Cookies",
    "Candy",
    "Pie",
    "Cake",
    "Fudge",
    "Dessert",
    "Biscuits",
    "Squares",
    "Brownies",
    "Scones",
    "Custard",
    "Cookie",
    "Cheesecake",
    "Icing",
    "Caramel",
    "Muffin",
    "Muffins",
    "Jello",
    "Butterscotch",
    "Breakfast",
    "Chocolate",
    "Muffins",
    "Crisp",
]

pattern = "|".join(keywords)

filtered_df = df[df["title"].str.contains(pattern, case=False, na=False)]

In [10]:
filtered_df

,Unnamed: 0,title,ingredients,directions,link,source,NER
0,0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu..."
4,4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...",www.cookbooks.com/Recipe-Details.aspx?id=659239,Gathered,"[""peanut butter"", ""graham cracker crumbs"", ""bu..."
6,6,Rhubarb Coffee Cake,"[""1 1/2 c. sugar"", ""1/2 c. butter"", ""1 egg"", ""...","[""Cream sugar and butter."", ""Add egg and beat ...",www.cookbooks.com/Recipe-Details.aspx?id=210288,Gathered,"[""sugar"", ""butter"", ""egg"", ""buttermilk"", ""flou..."
9,9,Millionaire Pie,"[""1 large container Cool Whip"", ""1 large can c...","[""Empty Cool Whip into a bowl."", ""Drain juice ...",www.cookbooks.com/Recipe-Details.aspx?id=794547,Gathered,"[""pineapple"", ""condensed milk"", ""lemons"", ""pec..."
11,11,Buckeye Candy,"[""1 box powdered sugar"", ""8 oz. soft butter"", ...","[""Mix sugar, butter and peanut butter."", ""Roll...",www.cookbooks.com/Recipe-Details.aspx?id=886785,Gathered,"[""powdered sugar"", ""butter"", ""peanut butter"", ..."
...,...,...,...,...,...,...,...
2231122,2231122,Breakfast Enchiladas,"[""1 ENCHILADA FILLING"", ""2 lb hot ground pork ...","[""Preheat oven to 350."", ""Cook sausage in a la...",cookpad.com/us/recipes/349310-breakfast-enchil...,Recipes1M,"[""ENCHILADA"", ""hot ground pork sausage"", ""butt..."
2231124,2231124,Chocolate Crumble,"[""6 1/2 tablespoons (94g) unsalted butter, sof...","[""Heat the oven to 375F or 350F on convection....",www.epicurious.com/recipes/food/views/chocolat...,Recipes1M,"[""unsalted butter"", ""cocoa"", ""flour"", ""sugar"",..."
2231125,2231125,Cran-Apple White Chocolate Popcorn,"[""12 ounces white chocolate chips"", ""12 teaspo...","[""Melt chocolate in large bowl set over pot of...",www.food.com/recipe/cran-apple-white-chocolate...,Recipes1M,"[""chocolate chips"", ""ground cinnamon"", ""ground..."
2231129,2231129,Baked Praline Ice Cream Cake,"[""1 cup packed brown sugar"", ""12 cup sour crea...","[""Set oven to 350 degrees /Grease a 13\"" x 9\""...",www.food.com/recipe/baked-praline-ice-cream-ca...,Recipes1M,"[""brown sugar"", ""sour cream"", ""butter"", ""butte..."


In [11]:
recipe_cleaned_df = df[~df.isin(filtered_df)].dropna()
recipe_cleaned_df

,Unnamed: 0,title,ingredients,directions,link,source,NER
1,1.0,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"[""beef"", ""chicken breasts"", ""cream of mushroom..."
2,2.0,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"[""frozen corn"", ""cream cheese"", ""butter"", ""gar..."
3,3.0,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,Gathered,"[""chicken"", ""chicken gravy"", ""cream of mushroo..."
5,5.0,Cheeseburger Potato Soup,"[""6 baking potatoes"", ""1 lb. of extra lean gro...","[""Wash potatoes; prick several times with a fo...",www.cookbooks.com/Recipe-Details.aspx?id=20115,Gathered,"[""baking potatoes"", ""extra lean ground beef"", ..."
7,7.0,Scalloped Corn,"[""1 can cream-style corn"", ""1 can whole kernel...","[""Mix together both cans of corn, crackers, eg...",www.cookbooks.com/Recipe-Details.aspx?id=876969,Gathered,"[""cream-style corn"", ""whole kernel corn"", ""cra..."
...,...,...,...,...,...,...,...
2231137,2231137.0,Sunny's Fake Crepes,"[""1/2 cup chocolate hazelnut spread (recommend...","[""Spread hazelnut spread on 1 side of each tor...",www.foodnetwork.com/recipes/sunny-anderson/sun...,Recipes1M,"[""chocolate hazelnut spread"", ""tortillas"", ""bu..."
2231138,2231138.0,Devil Eggs,"[""1 dozen eggs"", ""1 paprika"", ""1 salt and pepp...","[""Boil eggs on medium for 30mins."", ""Then cool...",cookpad.com/us/recipes/355411-devil-eggs,Recipes1M,"[""eggs"", ""paprika"", ""salt"", ""choice"", ""miracle..."
2231139,2231139.0,Extremely Easy and Quick - Namul Daikon Salad,"[""150 grams Daikon radish"", ""1 tbsp Sesame oil...","[""Julienne the daikon and squeeze out the exce...",cookpad.com/us/recipes/153324-extremely-easy-a...,Recipes1M,"[""radish"", ""Sesame oil"", ""White sesame seeds"",..."
2231140,2231140.0,Pan-Roasted Pork Chops With Apple Fritters,"[""1 cup apple cider"", ""6 tablespoons sugar"", ""...","[""In a large bowl, mix the apple cider with 4 ...",cooking.nytimes.com/recipes/1015164,Recipes1M,"[""apple cider"", ""sugar"", ""kosher salt"", ""bay l..."


In [12]:
len(filtered_df) / len(df)

0.2376330148417268

In [13]:
listoflist = recipe_cleaned_df["NER"].tolist()

In [14]:
array = recipe_cleaned_df["NER"].to_numpy()

In [15]:
array

array(['["beef", "chicken breasts", "cream of mushroom soup", "sour cream"]',
       '["frozen corn", "cream cheese", "butter", "garlic powder", "salt", "pepper"]',
       '["chicken", "chicken gravy", "cream of mushroom soup", "shredded cheese"]',
       ...,
       '["radish", "Sesame oil", "White sesame seeds", "Salt", "Soy sauce"]',
       '["apple cider", "sugar", "kosher salt", "bay leaves", "arbol", "berries", "caraway seeds", "mustard seeds", "coriander seeds", "pork chops", "Freshly ground black pepper", "neutral oil", "unsalted butter", "shallots", "thyme", "brandy", "heavy cream", "pork", "grain mustard", "horseradish", "apple cider", "cinnamon", "apples", "egg", "water", "rice flour", "all-purpose", "neutral oil"]',
       '["ground veal", "sausage", "bread crumbs", "milk", "garlic", "flat-leaf", "egg whites", "tomato paste", "freshly grated Pecorino Romano cheese", "Salt", "tomato sauce", "red pepper"]'],
      dtype=object)

In [16]:
array[0]

'["beef", "chicken breasts", "cream of mushroom soup", "sour cream"]'

In [17]:
len(array)

1700948

In [18]:
# Begin with an empty list
# Within each array, remove the quotations, remove the left and right brackets, and split into a list
# Add the cleaned array to the new list

cleaned_list = []

for i in range(len(array)):
    phrase = array[i]
    test2 = phrase.replace('"', "")
    test3 = test2.replace("[", "")
    test4 = test3.replace("]", "")
    phrase_to_list_cleaned = test4.split(",")
    cleaned_list.append(phrase_to_list_cleaned)

In [19]:
cleaned_list

[['beef', ' chicken breasts', ' cream of mushroom soup', ' sour cream'],
 ['frozen corn',
  ' cream cheese',
  ' butter',
  ' garlic powder',
  ' salt',
  ' pepper'],
 ['chicken', ' chicken gravy', ' cream of mushroom soup', ' shredded cheese'],
 ['baking potatoes',
  ' extra lean ground beef',
  ' butter',
  ' milk',
  ' salt',
  ' pepper',
  ' Cheddar cheese',
  ' bacon',
  ' green onion',
  ' sour cream'],
 ['cream-style corn',
  ' whole kernel corn',
  ' crackers',
  ' egg',
  ' butter',
  ' pepper'],
 ['tomatoes',
  ' water',
  ' onions',
  ' Worcestershire sauce',
  ' green peppers',
  ' oil'],
 ['dark sweet pitted cherries',
  ' ginger ale',
  ' flavor gelatin',
  ' boiling water',
  ' almond extract',
  ' marshmallows'],
 ['chicken', ' flour', ' barbecue sauce'],
 ['taco sauce',
  ' sour cream',
  ' cream cheese',
  ' ground beef',
  ' taco seasoning',
  ' shredded lettuce',
  ' tomato',
  ' onion',
  ' green pepper',
  ' Cheddar cheese'],
 ['broccoli',
  ' bacon',
  ' green on

In [20]:
len(cleaned_list)

1700948

In [21]:
# Remove the spaces in front of each word
cleaned_list_2 = [
    [item.strip().lower() for item in sub_list] for sub_list in cleaned_list
]

In [22]:
len(cleaned_list_2)

1700948

In [23]:
# Remove common household items from each ingredient list
cleaned_list_3 = [
    [item for item in sublist if item not in common_food_items]
    for sublist in cleaned_list_2
]

In [24]:
# Combine onions, onion --> onion
# Combine eggs, egg --> egg
# Combine tomato, tomatoes --> tomato
# Combine potato, potatoes --> potato
# Combine carrot, carrots --> carrot
# Combine mushroom, mushrooms --> mushroom, etc.


# Function to replace variations of onion in each inner list
def replace_onion(ingredients):
    ingredients = [re.sub("onions", "onion", ingredient) for ingredient in ingredients]
    ingredients = [re.sub("eggs", "egg", ingredient) for ingredient in ingredients]
    ingredients = [
        re.sub("tomatoes", "tomato", ingredient) for ingredient in ingredients
    ]
    ingredients = [
        re.sub("potatoes", "potato", ingredient) for ingredient in ingredients
    ]
    ingredients = [
        re.sub("red potato", "potato", ingredient) for ingredient in ingredients
    ]
    ingredients = [
        re.sub("baking potato", "potato", ingredient) for ingredient in ingredients
    ]
    ingredients = [
        re.sub("carrots", "carrot", ingredient) for ingredient in ingredients
    ]
    ingredients = [
        re.sub("mushrooms", "mushroom", ingredient) for ingredient in ingredients
    ]
    ingredients = [
        re.sub(
            r"\b(?:scallion|green\s*onion(?:ss)?|spring\s*onion)\b",
            "green onion",
            ingredient,
        )
        for ingredient in ingredients
    ]
    ingredients = [
        re.sub("scallions", "green onion", ingredient) for ingredient in ingredients
    ]
    ingredients = [
        re.sub("heavy cream", "cream", ingredient) for ingredient in ingredients
    ]
    ingredients = [
        re.sub("yellow onion", "onion", ingredient) for ingredient in ingredients
    ]
    ingredients = [
        re.sub("sweet onion", "onion", ingredient) for ingredient in ingredients
    ]
    ingredients = [
        re.sub("white onion", "onion", ingredient) for ingredient in ingredients
    ]
    ingredients = [
        re.sub("stalks celery", "celery", ingredient) for ingredient in ingredients
    ]
    ingredients = [
        re.sub("breadcrumbs", "bread crumbs", ingredient) for ingredient in ingredients
    ]
    ingredients = [
        re.sub("shallots", "shallot", ingredient) for ingredient in ingredients
    ]
    ingredients = [
        re.sub("cucumbers", "cucumber", ingredient) for ingredient in ingredients
    ]
    ingredients = [
        re.sub("cream ofchicken soup", "cream of chicken soup", ingredient)
        for ingredient in ingredients
    ]
    ingredients = [
        re.sub("fresh pasta", "pasta", ingredient) for ingredient in ingredients
    ]
    ingredients = [
        re.sub("frozen peas", "peas", ingredient) for ingredient in ingredients
    ]
    ingredients = [
        re.sub("oranges", "orange", ingredient) for ingredient in ingredients
    ]
    ingredients = [
        re.sub(r"\bapples\b", "apple", ingredient) for ingredient in ingredients
    ]  # inserted by ecb convert to apple
    ingredients = [
        re.sub("cream of mushroom soup", "mushroom soup", ingredient)
        for ingredient in ingredients
    ]
    ingredients = [
        re.sub("bananas", "banana", ingredient) for ingredient in ingredients
    ]
    ingredients = [
        re.sub("egg noodles", "noodles", ingredient) for ingredient in ingredients
    ]
    ingredients = [
        re.sub("avocados", "avocado", ingredient) for ingredient in ingredients
    ]
    ingredients = [
        re.sub("whole kernel corn", "corn", ingredient) for ingredient in ingredients
    ]
    ingredients = [
        re.sub("white bread", "bread", ingredient) for ingredient in ingredients
    ]
    ingredients = [
        re.sub(r"\b(?:mandarin\s*oranges?|mandarin\s*orange)\b", "orange", ingredient)
        for ingredient in ingredients
    ]
    ingredients = [
        re.sub("orange zest", "orange", ingredient) for ingredient in ingredients
    ]
    ingredients = [
        re.sub("fresh mushroom", "mushroom", ingredient) for ingredient in ingredients
    ]
    ingredients = [
        re.sub("elbow macaroni", "macaroni", ingredient) for ingredient in ingredients
    ]
    ingredients = [
        re.sub("baking potato", "potato", ingredient) for ingredient in ingredients
    ]
    ingredients = [
        re.sub("plain yogurt", "yogurt", ingredient) for ingredient in ingredients
    ]
    ingredients = [re.sub("choy", "bok choy", ingredient) for ingredient in ingredients]
    ingredients = [
        re.sub(r"\b(?:mozzarella|parmesan|cheddar)\b", "cheese", ingredient)
        for ingredient in ingredients
    ]
    ingredients = [
        re.sub(r"\b(?:flour|corn)\s*tortilla(?:s)?\b", "tortilla", ingredient)
        for ingredient in ingredients
    ]
    ingredients = [
        re.sub(
            r"\b(?:fresh|baby|frozen)?\s*spinach(?:\s*leaves)?\b", "spinach", ingredient
        )
        for ingredient in ingredients
    ]
    ingredients = [
        re.sub(
            r"\b(?:frozen\s*)?(?:broccoli\s*florets|broccoli)\b", "broccoli", ingredient
        )
        for ingredient in ingredients
    ]
    ingredients = [
        re.sub(r"\b(?:white|brown|long\s*grain)?\s*rice\b", "rice", ingredient)
        for ingredient in ingredients
    ]
    ingredients = [
        re.sub(r"\b(?:frozen|fresh)?\s*strawberries\b", "strawberries", ingredient)
        for ingredient in ingredients
    ]
    ingredients = [
        re.sub(
            r"\b(?:hamburg|hamburger|lean\s*ground\s*beef)\b", "ground beef", ingredient
        )
        for ingredient in ingredients
    ]
    return ingredients


# Apply the function to the inner list of recipes
cleaned_list_4 = [replace_onion(recipe) for recipe in cleaned_list_3]

In [25]:
# Combine chicken and chicken breasts --> chicken


# Function to replace variations of chicken in each inner list
def replace_chicken(ingredients):
    return [
        re.sub(r"\b(chicken breasts?| chicken)\b", "chicken", ingredient)
        for ingredient in ingredients
    ]


# Apply the function to the inner list of recipes
cleaned_list_5 = [replace_chicken(recipe) for recipe in cleaned_list_4]

In [26]:
# Combine lemon, lemons, lemon juice, lemon zest ---> lemon


# Function to replace variations of lemon in each inner list
def replace_lemon(ingredients):
    return [
        re.sub(r"\b(lemon(s| juice| zest)?)\b", "lemon", ingredient)
        for ingredient in ingredients
    ]


# Apply the function to the inner list of recipes
cleaned_list_6 = [replace_lemon(recipe) for recipe in cleaned_list_5]

In [27]:
# Combine lime, limes, lime juice --> lime


# Function to replace variations of lime in each inner list
def replace_lime(ingredients):
    return [
        re.sub(r"\b(lime(s| juice| zest)?)\b", "lime", ingredient)
        for ingredient in ingredients
    ]


# Apply the function to the inner list of recipes
cleaned_list_7 = [replace_lime(recipe) for recipe in cleaned_list_6]

In [28]:
def replace_cheese(recipe):
    cheese_pattern = re.compile(
        r"\b(?:cheddar|parmesan|mozzarella|swiss|grated|feta|velveeta|ricotta|parmesan|american|goat|shredded|blue|romano|percorino|provolone|shredded monterey jack)\s*cheese\b",
        re.IGNORECASE,
    )

    # Use re.sub to replace matching cheese types with "cheese"
    recipe = [cheese_pattern.sub("cheese", ingredient) for ingredient in recipe]

    return recipe


cleaned_list_8 = [replace_cheese(recipe) for recipe in cleaned_list_7]

In [29]:
# Bell pepper data cleaning

bell_pepper_variations = [
    "green bell bell peppers",
    "greeen bell peppers",
    "green peppers",
    "green bell bell pepper",
    "green bell pepper",
    "green pepper",
    "red bell bell peppers",
    "red bell peppers",
    "red peppers",
    "red bell bell pepper",
    "red bell pepper",
    "red pepper",
    "orange bell bell peppers",
    "orange bell peppers",
    "orange peppers",
    "orange bell bell pepper",
    "orange bell pepper",
    "orange pepper",
    "yellow bell bell peppers",
    "yellow bell peppers",
    "yellow peppers",
    "yellow bell bell pepper",
    "yellow bell pepper",
    "yellow pepper",
    "bell bell peppers",
    "bell peppers",
    "peppers",
    "bell bell pepper",
    "sweet bell pepper",
    "sweet bell peppers",
]

cleaned_list_9 = [
    ["bell pepper" if item in bell_pepper_variations else item for item in sublist]
    for sublist in cleaned_list_8
]

In [30]:
# Fox cheese cheese error


def replace_cheese_cheese(ingredients):
    ingredients = [
        re.sub("cheese cheese", "cheese", ingredient) for ingredient in ingredients
    ]
    return ingredients


# Apply the function to the inner list of recipes
cleaned_list_10 = [replace_cheese_cheese(recipe) for recipe in cleaned_list_9]

In [31]:
# UPDATE!!
cleaned_list_10

[['beef', 'chicken', 'mushroom soup', 'sour cream'],
 ['frozen corn', 'cream cheese'],
 ['chicken', 'chicken gravy', 'mushroom soup', 'cheese'],
 ['potato',
  'extra ground beef',
  'milk',
  'cheese',
  'bacon',
  'green onion',
  'sour cream'],
 ['cream-style corn', 'corn', 'crackers', 'egg'],
 ['tomato', 'onion', 'bell pepper'],
 ['dark sweet pitted cherries',
  'ginger ale',
  'flavor gelatin',
  'almond extract'],
 ['chicken'],
 ['taco sauce',
  'sour cream',
  'cream cheese',
  'ground beef',
  'taco seasoning',
  'shredded lettuce',
  'tomato',
  'onion',
  'bell pepper',
  'cheese'],
 ['broccoli', 'bacon', 'green onion', 'raisins'],
 ['strawberries', 'pineapple', 'banana', 'pecans', 'strawberry', 'sour cream'],
 ['egg', 'soda', 'nuts', 'banana'],
 [],
 ['tomato', 'tomato paste', 'bell pepper', 'sweet basil'],
 ['ground beef', 'tomato juice', 'oats', 'egg', 'onion'],
 ['steak',
  'ground white pepper',
  'bell pepper',
  'beef stock',
  'onion',
  'celery'],
 ['long', 'tomato sa

In [32]:
# Count length of each recipe; store count and continue to add to it
# Divide by 1700949

sum_of_list = 0

for i in range(len(cleaned_list_4)):
    item = len(cleaned_list_4[i])
    sum_of_list += item

In [33]:
average_num_ingredients = sum_of_list / len(cleaned_list)
average_num_ingredients

4.863217452855702

In [34]:
def flatten_comprehension(matrix):
    return [item for row in matrix for item in row]

In [35]:
# UPDATE!!
complete_ingredient_list = flatten_comprehension(cleaned_list_10)

In [36]:
complete_ingredient_list

['beef',
 'chicken',
 'mushroom soup',
 'sour cream',
 'frozen corn',
 'cream cheese',
 'chicken',
 'chicken gravy',
 'mushroom soup',
 'cheese',
 'potato',
 'extra ground beef',
 'milk',
 'cheese',
 'bacon',
 'green onion',
 'sour cream',
 'cream-style corn',
 'corn',
 'crackers',
 'egg',
 'tomato',
 'onion',
 'bell pepper',
 'dark sweet pitted cherries',
 'ginger ale',
 'flavor gelatin',
 'almond extract',
 'chicken',
 'taco sauce',
 'sour cream',
 'cream cheese',
 'ground beef',
 'taco seasoning',
 'shredded lettuce',
 'tomato',
 'onion',
 'bell pepper',
 'cheese',
 'broccoli',
 'bacon',
 'green onion',
 'raisins',
 'strawberries',
 'pineapple',
 'banana',
 'pecans',
 'strawberry',
 'sour cream',
 'egg',
 'soda',
 'nuts',
 'banana',
 'tomato',
 'tomato paste',
 'bell pepper',
 'sweet basil',
 'ground beef',
 'tomato juice',
 'oats',
 'egg',
 'onion',
 'steak',
 'ground white pepper',
 'bell pepper',
 'beef stock',
 'onion',
 'celery',
 'long',
 'tomato sauce',
 'onion',
 'ground bee

In [37]:
# Create a unique list of ingredients

set_ingredients = set(complete_ingredient_list)
list_unique_ingredients = list(set_ingredients)

In [38]:
len(list_unique_ingredients)

168683

In [39]:
word_counts = Counter(complete_ingredient_list)

In [40]:
sorted_ingredient_counts = sorted(word_counts.items(), key=lambda x: x[1], reverse=True)

In [41]:
sorted_ingredient_counts

[('onion', 511747),
 ('cheese', 394539),
 ('egg', 303922),
 ('bell pepper', 232745),
 ('milk', 222580),
 ('tomato', 202551),
 ('chicken', 166520),
 ('celery', 130640),
 ('green onion', 113823),
 ('carrot', 107471),
 ('sour cream', 96909),
 ('potato', 95162),
 ('ground beef', 93339),
 ('mushroom', 74568),
 ('cream cheese', 74519),
 ('lemon', 72501),
 ('rice', 67860),
 ('bacon', 67832),
 ('lime', 59953),
 ('pineapple', 52560),
 ('cream', 52535),
 ('bread crumbs', 51836),
 ('red onion', 50228),
 ('bread', 46251),
 ('mushroom soup', 44711),
 ('tomato sauce', 41496),
 ('white wine', 41189),
 ('orange', 38280),
 ('pecans', 38135),
 ('shrimp', 36648),
 ('shallot', 36295),
 ('nuts', 36234),
 ('orange juice', 35986),
 ('broccoli', 35955),
 ('zucchini', 35237),
 ('corn', 34502),
 ('cucumber', 32987),
 ('spinach', 32036),
 ('cream ofchicken soup', 31496),
 ('banana', 30821),
 ('apple', 30406),
 ('tomato paste', 29349),
 ('walnuts', 27473),
 ('raisins', 26319),
 ('almonds', 25790),
 ('avocado', 23

In [45]:
pd.set_option("display.max_rows", 110)
sorted_ingredient_counts_df = pd.DataFrame(sorted_ingredient_counts[:100])
sorted_ingredient_counts_df.to_csv("top_100_sorted_ingredient_counts.csv")
sorted_ingredient_counts_df

,0,1
0,onion,511747
1,cheese,394539
2,egg,303922
3,bell pepper,232745
4,milk,222580
5,tomato,202551
6,chicken,166520
7,celery,130640
8,green onion,113823
9,carrot,107471


In [46]:
# Add cleaned_list_9 back into the dataframe --> recipe_cleaned_df

recipe_cleaned_df["cleaned_NER"] = cleaned_list_10

In [47]:
recipe_cleaned_df.drop("Unnamed: 0", axis=1, inplace=True)

In [48]:
recipe_cleaned_df

,title,ingredients,directions,link,source,NER,cleaned_NER
1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"[""beef"", ""chicken breasts"", ""cream of mushroom...","[beef, chicken, mushroom soup, sour cream]"
2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"[""frozen corn"", ""cream cheese"", ""butter"", ""gar...","[frozen corn, cream cheese]"
3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,Gathered,"[""chicken"", ""chicken gravy"", ""cream of mushroo...","[chicken, chicken gravy, mushroom soup, cheese]"
5,Cheeseburger Potato Soup,"[""6 baking potatoes"", ""1 lb. of extra lean gro...","[""Wash potatoes; prick several times with a fo...",www.cookbooks.com/Recipe-Details.aspx?id=20115,Gathered,"[""baking potatoes"", ""extra lean ground beef"", ...","[potato, extra ground beef, milk, cheese, baco..."
7,Scalloped Corn,"[""1 can cream-style corn"", ""1 can whole kernel...","[""Mix together both cans of corn, crackers, eg...",www.cookbooks.com/Recipe-Details.aspx?id=876969,Gathered,"[""cream-style corn"", ""whole kernel corn"", ""cra...","[cream-style corn, corn, crackers, egg]"
...,...,...,...,...,...,...,...
2231137,Sunny's Fake Crepes,"[""1/2 cup chocolate hazelnut spread (recommend...","[""Spread hazelnut spread on 1 side of each tor...",www.foodnetwork.com/recipes/sunny-anderson/sun...,Recipes1M,"[""chocolate hazelnut spread"", ""tortillas"", ""bu...","[chocolate hazelnut spread, tortillas, hazelnuts]"
2231138,Devil Eggs,"[""1 dozen eggs"", ""1 paprika"", ""1 salt and pepp...","[""Boil eggs on medium for 30mins."", ""Then cool...",cookpad.com/us/recipes/355411-devil-eggs,Recipes1M,"[""eggs"", ""paprika"", ""salt"", ""choice"", ""miracle...","[egg, choice, miracle whip, relish]"
2231139,Extremely Easy and Quick - Namul Daikon Salad,"[""150 grams Daikon radish"", ""1 tbsp Sesame oil...","[""Julienne the daikon and squeeze out the exce...",cookpad.com/us/recipes/153324-extremely-easy-a...,Recipes1M,"[""radish"", ""Sesame oil"", ""White sesame seeds"",...","[radish, sesame oil, white sesame seeds]"
2231140,Pan-Roasted Pork Chops With Apple Fritters,"[""1 cup apple cider"", ""6 tablespoons sugar"", ""...","[""In a large bowl, mix the apple cider with 4 ...",cooking.nytimes.com/recipes/1015164,Recipes1M,"[""apple cider"", ""sugar"", ""kosher salt"", ""bay l...","[apple cider, arbol, berries, caraway seeds, m..."


In [49]:
# Write sorted_ingredient_counts to a csv


# Specify the file path
file_path = "ingredient_dictionary_v3.csv"

# Open the file in write mode ('w')
with open(file_path, "w", newline="") as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerows(sorted_ingredient_counts)


# Confirmation message
print(f"The list has been written to {file_path}")

The list has been written to ingredient_dictionary_v3.csv


In [50]:
# WRITE UNIQUE INGREDIENT LIST TO A TEXT FILE

# Specify the file path
file_path = "ingredient_list_v7.txt"

# Open the file in write mode ('w')
with open(file_path, "w") as file:
    for item in list_unique_ingredients:
        file.write(str(item) + "\n")


# Confirmation message
print(f"The list has been written to {file_path}")

The list has been written to ingredient_list_v7.txt


In [51]:
# Write the updated recipelist to a dataframe

#'RecipeNLG_dataset.csv'

filename = "recipe_dataset_cleaned_v4.csv"

recipe_cleaned_df.to_csv(filename, index=False)

print("CSV file written successfully")

CSV file written successfully
